# The SQL database for Stellab

Towards a stellab widget interface, similar to SYGMA and OMEGA widgets.

SQL code to create stellab database. The table structure is shown below.

Tables:
* refs: Table with literature references:
    * refid: unique reference id (PK)
    * fauthor: first author
    * year
    * nasaads: link to nasa ads page

* galaxies
    * gali: unique ref id
    * name: galaxy name

* solarnorms
    * normid: unique reference id (PK)
    * H, He... abundance entries
    * refids
* abu_table_reg
    * abu_reg_id uniqure reference
    * refid to look up literature
    * table name to look up abundance table
    * normid to look up normalization used

* multiple abundance tables, all with
    * abuid: unique reference id
    * element ratio1, element ratio2...
    * error1, errror2 ...

Rules:
* Internal connection only via (private keys) PKs.
* On user level no interaction with PKs.
* As such table entries (except PKs) can be easily modified without allowing any disconnections between tables.

Tips:
* Use not null
* fixed lenght type arrays are faster

In [5]:
#We choose sqlite3 because it does not require to install mysql or run a server.
import sqlite3
import pandas as pd
import stellab as st

SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. 

Creating database and table refs

### Add references

In [6]:
st1=st.stellab()
#papers=st1.list_ref_papers()
legs=st1.leg

name=[]
year=[]
for i,labels in enumerate(legs):
    name.append(labels.split('(')[0].strip())
    year.append(labels.split('(')[1].strip()[:-1])

In [7]:
st1.list_solar_norm()

Anders_Grevesse_1989
Grevesse_Noels_1993
Grevesse_Sauval_1998
Asplund_et_al_2009
Asplund_et_al_2005
Lodders_et_al_2009


## I. Code to build database

In [8]:
#create table
!rm stellab.db
db=sqlite3.connect('stellab.db')
curser=db.cursor()
curser.execute('''CREATE TABLE IF NOT EXISTS refs (refid integer primary key autoincrement, fauthor text, year year, nasads text)''')
curser.execute('''CREATE TABLE IF NOT EXISTS galaxies (galid integer primary key autoincrement, gal_name text)''')
curser.execute('''CREATE TABLE IF NOT EXISTS solarnorms (normid integer primary key autoincrement, H float, He float, refid integer)''')
#curser.execute('''CREATE TABLE IF NOT EXISTS abu_table_reg (abu_reg_id integer primary key autoincrement, refid integer, normid integer,abutable text, gal_name)''')
#the abu_table_reg has a foreign key referencing galaxies table
curser.execute('''CREATE TABLE IF NOT EXISTS abu_table_reg (abu_reg_id integer primary key autoincrement, refid integer, normid integer,abutable text,galid integer, FOREIGN KEY(galid) REFERENCES galaxies(galid))''')

db.commit()

In [9]:
def get_column_names(table):
    tmp=db.execute("PRAGMA table_info(%s)" % table).fetchall()
    return [entry[1] for entry in tmp]

***
### Routines for adding data.

In [10]:
def check_solar_normalization(name,year):
    '''
    Check if normalization data is available in table solarnorms. Only for internal use.
    '''        
    #check if paper for normalization data exists already.
    results = db.execute('''SELECT refid FROM refs WHERE (fauthor = '%s' and year = '%s' )''' % (name,year)).fetchall()  #% ('Anders','1989'))
    refid = results
    if len(refid)==1:
        refid = refid[0][0]
        results = db.execute('''SELECT normid FROM solarnorms WHERE (refid = %s )''' %(refid))
        normid = results.fetchall()
        if len(normid)==1:
            print 'found corresponding normalization data.'
            return normid[0][0]
        else:
            #add normalization data
            print 'normalization data is missing. Add the data first by using add_normalization().'
            return -1
    else:
        print 'Solar normalization data is not in database. Add the data first by using add_normalization().'
        return -1

In [11]:
def check_galaxy(galaxy_name):
    results = db.execute('''SELECT galid FROM galaxies WHERE (gal_name = '%s') ''' % galaxy_name).fetchall()
    #if galaxy does not exist
    if len(results)==0:
        galid = -1
    else:
        galid = results[0][0]
    return galid    

In [12]:
def add_solar_normalization(norm_paper,norm_label,norm_data):
    '''
    Add data to normalization table solarnorms and the corresponding paper information to table refs, if necessary.
    E.g.
    norm_paper=['Venn',2012,'http://adsabs.harvard.edu/abs/2012ApJ...751..102V']
    norm_label=['H','He']
    norm_data = [-2.81,0.34,]

    '''
    #check if paper for normalization data exists in table refs.
    name=norm_paper[0]
    year=norm_paper[1]
    nasads=norm_paper[2]
    results = db.execute('''SELECT refid FROM refs WHERE (fauthor = '%s' and year = '%s' )''' % (name,year)).fetchall()  #% ('Anders','1989'))
    refid = results
    #if paper is not available in  refs, add it
    if len(refid)==0:
        print 'add paper for solar normalization to table refs.'
        add_paper_ref(name,year,nasads)
        results = db.execute('''SELECT refid FROM refs WHERE (fauthor = '%s' and year = '%s' )''' % (name,year)).fetchall()   #% ('Anders','1989'))
        #get PK refid for paper entry
        refid = results               
    refid = refid[0][0]
    
    #check for normalization data in table solarnorms, if not available, add it.
    results = db.execute('''SELECT normid FROM solarnorms WHERE (refid = %s )''' %(refid)).fetchall()
    normid = results
    if len(normid)==1:
        print 'normalization data is already available.'
    else:         
        #check if columns of solarnorms table include all entries of norm_label. If not, add new columns.
        columns= get_column_names('solarnorms') 
        for k in range(len(norm_label)):
            if not norm_label[k] in columns:
                #print norm_label[k],'missing in columns, add new column'
                db.execute('''ALTER TABLE solarnorms ADD %s float''' %norm_label[k])                  
        #add normalization data
        str_tmp = 'refid, '
        for k in range(len(norm_label)):
                str_tmp+= (norm_label[k] + ',')
        str_tmp = str_tmp[:-1]
        sql_prepr = tuple([str_tmp])
        str_tmp = str(refid)+', '
        for k in range(len(norm_data)):
                str_tmp+= str(norm_data[k]) + ','
        str_tmp = str_tmp[:-1]
        sql_prepr = sql_prepr + tuple([str_tmp])
        db.execute('''INSERT INTO solarnorms (%s) VALUES (%s)''' % sql_prepr)
        db.commit()

In [13]:
def add_paper_ref(name,year,nasads):
    '''
    Add a new paper entry to the table refs, if it does not exist yet. Return PK ref_id of table refs.
    e.g.
    name='Anders'
    year=1989
    nasads='http://ukads.nottingham.ac.uk/abs/1993A%26A...271..587G'
    '''   
    results = db.execute('''SELECT refid FROM refs WHERE (fauthor = '%s' and year = '%s' )''' % 
                         (name,year)).fetchall()
    if len(results)==0:
            print 'add paper related to abundance to table refs.'
            db.execute('''INSERT INTO refs (fauthor, year,nasads) VALUES ('%s','%s','%s') ''' % (name,year,nasads))
            db.commit()
            results=db.execute('''SELECT refid FROM refs WHERE (fauthor='%s' and year='%s') ''' % (name,year)).fetchall()
            ref_id=results[0][0]
    else:
        print 'abu data paper exists already. do nothing'
        ref_id = results[0][0]
    return ref_id

In [14]:
def add_galaxy(galaxy_name):
    results = db.execute('''SELECT galid FROM galaxies WHERE (gal_name = '%s') ''' % galaxy_name).fetchall()
    #galaxy does not exist
    if len(results)==0:
        print 'galaxy name ',galaxy_name
        db.execute('''INSERT INTO galaxies (gal_name) VALUES ('%s') ''' % (galaxy_name))
        db.commit()
        results=db.execute('''SELECT galid FROM galaxies WHERE (gal_name= '%s') ''' % (galaxy_name)).fetchall()
        galid = results[0][0]
    else:
        galid = results[0][0]
    return galid

In [15]:
def add_abundance_data(abu_paper,abu_norm,abu_label,abu_data):
    '''
    Adding new abundance data to database. Creates new table. Requires corresponding solar normalization to be available
    in database.
    e.g.
    abundance_paper=['Venn',2012,'http://adsabs.harvard.edu/abs/2012ApJ...751..102V','Milky Way']
    abundance_norm=['Anders','1989']
    abundance_label=['[Fe/H]','err']
    abundance_data = [-2.81,0.34]    
    '''
    
    ######check if solar normalization paper and data already exists in database
    #get PK norm_id for normalization table, if not available return -1
    norm_id = check_solar_normalization(abu_norm[0],abu_norm[1])
    if norm_id == -1: return
    
    ######check if abundance paper into database, if it does not exist, add it.
    #get PK ref_id for refs table entry
    ref_id = add_paper_ref(abu_paper[0],abu_paper[1],abu_paper[2])

    ######check galaxy name, if it does not exist add it.
    gal_name = abundance_paper[3]
    #get PK galid for galaxy table entry
    galid=add_galaxy(gal_name)
    print 'galid ',galid
    
    ###### add abundance data    
    table_name = ''
    #check if table already exists in registry table for abundance tables
    results = curser.execute('''SELECT ar.abu_reg_id FROM abu_table_reg ar WHERE (normid = '%s' and refid = '%s' and galid = '%s')''' % (norm_id,ref_id,galid)).fetchall()
    if len(results)==0:
        
        #get the latest key entry
        abu_reg_ids = curser.execute('''SELECT abu_reg_id FROM abu_table_reg''').fetchall()
        if len(abu_reg_ids)==0:
            abu_reg_id = 0
        else:
            abu_reg_id = abu_reg_ids[-1][0]
        #create new key entry
        abu_reg_id = abu_reg_id + 1
        
        #name table according to numbering of abu_reg_id
        tablename = 'abu_table_%s' % abu_reg_id
        print 'create table ',tablename
        #add new table
                             
        #create table
        str_tmp=''
        for k in range(len(abu_label)):
            str_tmp+=' , '+abu_label[k] +' float '
                             
        sql_prepr = tuple([tablename])+tuple([str_tmp])
                             
        #create database
        #print 'create abu table entry'
        curser.execute('''CREATE TABLE IF NOT EXISTS %s (abuid integer primary key autoincrement %s )''' %
                            sql_prepr)
                             
        #### add abundance data: 1 entry line        
        str_tmp1=''
        for k in range(len(abu_label)):
                str_tmp1+=abu_label[k]+','
        str_tmp1 = str_tmp1[:-1]
        str_tmp2=''
        for k in range(len(abu_data)):
                str_tmp2+=str(abu_data[k])+','
        str_tmp2 = str_tmp2[:-1]        
        
        #create tuple for SQL input
        sql_prepr = tuple([tablename]) + tuple([str_tmp1]) + tuple([str_tmp2])
        #print sql_prepr           
        curser.execute('''INSERT INTO %s (%s) VALUES (%s)''' %sql_prepr)
        
        #insert into table registry last, after abundance table was created successfully.                     
        curser.execute('''INSERT INTO abu_table_reg (refid, normid,abutable,galid) VALUES ('%s','%s','%s','%s') ''' %
                             (ref_id,norm_id,tablename,galid))
        #print 'Table ',sql_prepr,' , ',' created!'
    else:
        print 'abundance table with same paper reference and solar normalization exists already! Do nothing.'

***
### Routines for retrieving data.

In [16]:
# style of pandas sheet to allow nasads link to be clickable
def make_clickable(val):
    '''
    internal function for pandas display
    '''
    if 'http' in str(val):
        return '<a href="{}">{}</a>'.format(val,val)
    else:
        return val

In [17]:
def get_solar_normalizations(norm_paper=[],data_x_y=False):
    '''
    Access either specific solar normalization data, specified through norm_paper or all solar normalization
    data when norm_paper=[].
    e.g. norm_paper=['Anders',1989]
    '''
    
    all_data= db.execute('''SELECT sn.*,r.fauthor,r.year,r.nasads FROM solarnorms sn INNER JOIN refs as r ON sn.refid = r.refid''').fetchall()
    columns = get_column_names('solarnorms')
    columns=columns + ['fauthor','year','nasaads']
 
    if not len(norm_paper) == 0:
        idx=-1
        for k in range(len(all_data)):
            print all_data[k]
            if norm_paper[0] in all_data[k] and norm_paper[1] in all_data[k]:
                idx=k
                break
        if idx == -1: return 'normalization table not found.'
        data = [all_data[idx]]
    else:
        data = all_data
    if data_x_y:
        return columns,data        
    else:    
        return pd.DataFrame(data=data,columns=columns).style.format(make_clickable)

In [18]:
def get_paper_refs():
    '''
    Access all the paper references available in the database.
    '''
    data= db.execute('''SELECT * FROM refs''').fetchall()
    columns = get_column_names('refs')
    return pd.DataFrame(data=data,columns=columns).style.format(make_clickable)

In [19]:
def get_galaxies(data_x_y=False):
    data = db.execute('''SELECT gal_name FROM galaxies''').fetchall()
    columns = ['Galaxies']    
    if data_x_y:
        return columns,data
    
    else:
        return pd.DataFrame(data=data,columns=columns) 

In [20]:
def get_overview_abundance_tables(data_x_y=False):
    '''
    Overview over all available abundance tables. 
    '''
    data= db.execute('''SELECT a.*,r.fauthor,r.year,r.nasads FROM abu_table_reg a INNER JOIN refs r ON a.refid = r.refid''').fetchall()
    columns = get_column_names('abu_table_reg')
    columns=columns + ['fauthor','year','nasaads']
    if data_x_y:
        return columns,data
    
    else:
        return pd.DataFrame(data=data,columns=columns).style.format(make_clickable)

In [21]:
def get_abundance_data(abu_paper,data_x_y=False):
    '''
    Access abundance data from specific paper abu_paper
    e.g. abu_paper=['Venn',2012,'Milky Way']   
    '''
    fname = abu_paper[0]
    year = abu_paper[1]
    galaxy_name= abu_paper[2]
    
    #check for availability of galaxy_name
    galid=check_galaxy(galaxy_name)
    if galid==-1: return 'galaxy name not available.'
    
    #check for availability, instead of sql query
    columns,all_data = get_overview_abundance_tables(data_x_y=True)
    
    idx=-1
    for k in range(len(all_data)):
        if ((fname in all_data[k] and year in all_data[k]) and (galid in all_data[k])):
            idx=k
            break
    if idx==-1: return 'abundance table not found in database.'
    data= all_data[idx]
    #print columns
    idx=columns.index('abutable')
    tablename = data[idx]
    #get the abundance data from specific table tablename
    data= db.execute('''SELECT * FROM %s ''' % tablename).fetchall()
    print data
    columns=get_column_names(tablename)
    if data_x_y:
        return columns,data
    else:
        return pd.DataFrame(data=data,columns=columns)   

***
### Routines for updating existing data

In [22]:
def update_paper_refs(refid,**kwargs):
    '''
    Update table entry selectd by refid of paper reference table and providing arguments
    fauthor, year, nasads
    '''
    all_columns=get_column_names('refs')
    value_declaration = ''
    for k,column in enumerate(kwargs):
        print k,column
        if not column in all_columns:
            print 'Column argument ',column,'is not available'
            return 
        value_declaration += str(column)+'=' + "'" + str(kwargs[column])+"',"
    value_declaration = value_declaration[:-1]
    print value_declaration
    db.execute('''UPDATE refs SET %s WHERE (refid = %s)''' % (value_declaration,refid))
    db.commit()

In [23]:
def update_solar_normalizations(normid,**kwargs):
    '''
    Update table entry selectd by normid of solar normalization table and providing arguments
    available as columns in the table.
    '''
    
    all_columns_solarnorms=get_column_names('solarnorms')
    all_columns_refs=get_column_names('refs')
    
    #check if update requires updating table refs
    kwargs_solarnorms={}
    kwargs_refs={}
    for k,column in enumerate(kwargs):
        if column in all_columns_refs:
            print 'Column value ',column,' belongs to paper reference table. Update value via update_paper_refs()'
            return
        else:
            kwargs_solarnorms[column] = kwargs[column]
        
    value_declaration = ''
    for k,column in enumerate(kwargs_solarnorms):
        print k,column
        if not column in all_columns_solarnorms:
            print 'Column argument ',column,'is not available'
            return 
        value_declaration += str(column)+'=' + "'" + str(kwargs_solarnorms[column])+"',"
    value_declaration = value_declaration[:-1]
    print value_declaration
    db.execute('''UPDATE solarnorms SET %s WHERE (normid = %s)''' % (value_declaration,normid))
    db.commit()

I refraid from implementing an update capability for the abundance table because these cannot be checked by hand. Instead I recommend deleting the whole table.

In [24]:
def update_abundance_data(abu_paper,abu_norm,abu_label,abu_data):
    '''
    '''
    #delete table
    
    #add abundance table again.
    add_abundance_data(abu_paper,abu_norm,abu_label,abu_data)


## II. Examples how to add and retrieve data.

#### Before we add abundance data we need to add a related solar normalization data and (paper) reference. The solar normalization data goes in one table (solarnorms table) and the paper reference in another (refs table).

In [25]:
#add normalization data
normalization_paper=['Anders',1989,'http://ukads.nottingham.ac.uk/abs/1993A%26A...271..587G']
normalization_label=['H','He','Fe']
normalization_data = [-2.81,0.34,0.01]
add_solar_normalization(normalization_paper,normalization_label,normalization_data)

add paper for solar normalization to table refs.
add paper related to abundance to table refs.


check that table with solar noramlization data is correctly populated:

In [26]:
#did we successfully created database entry in solar normalization table?
get_solar_normalizations()

Lets update an entry of the normalization table.

In [27]:
update_solar_normalizations(normid=1,He=99)

0 He
He='99'


In [28]:
get_solar_normalizations()

In [29]:
update_solar_normalizations(normid=1,fauthor='Mr. X')

Column value  fauthor  belongs to paper reference table. Update value via update_paper_refs()


Check if table with paper references is correctly updated:

In [30]:
#lets see if there is an entry for the paper referene in the refs table
get_paper_refs()

In [31]:
#update_paper=['Anders',1989,'http://ukads.nottingham.ac.uk/abs/1993A%26A...271..587G']
update_paper_refs(refid=3,nasads='http://ukads.nottingham.ac.uk/abs/1993A%26A...271..587G',year=92999)

0 nasads
1 year
nasads='http://ukads.nottingham.ac.uk/abs/1993A%26A...271..587G',year='92999'


In [32]:
get_paper_refs()

#### Add one abundance data table. A new table with abundance data is created and a new entry is added to the abu_table_reg table. The latter table holds the names and info about all abundance data tables available.

In [35]:
#add abundance data
abundance_paper=['Venn',2012,'http://adsabs.harvard.edu/abs/2012ApJ...751..102V','Milky Way']
#corresponding normalization for abundance data. Must be existant in database, else use add_solar_normalization() above.
abundance_norm=['Anders','1989']
#plain data
abundance_label=['[Fe/H]','err']
abundance_data = [-2.81,0.34]
add_abundance_data(abundance_paper,abundance_norm,abundance_label,abundance_data)

found corresponding normalization data.
abu data paper exists already. do nothing
galid  1
abundance table with same paper reference and solar normalization exists already! Do nothing.


In [38]:
#check if table exists, and print it.
abundance_paper=['Venn',2012,'Milky Way']
get_abundance_data(abu_paper=abundance_paper)

[(1, -2.81, 0.34)]


,abuid,Fe/H,err
0,1,-2.81,0.34


In [39]:
#check if table exists in the registration table abu_table_reg.
#refid is the id which refers to the paper reference in the refs table which holds all references.
get_overview_abundance_tables()

In [40]:
#lets see the refs table. Indeed the refid of 2 refers to the Venn et al paper.
get_paper_refs()

In [41]:
get_galaxies()

,Galaxies
0,Milky Way


#### Example to add second and third abundance data tables

In [42]:
#add abundance data
abundance_paper=['BeersFun',2008,'http://adsabs.harvard.edu/abs/2012ApJ...751..102V','Milky Way']
abundance_norm=['Anders','1989']
abundance_label=['[Fe/H]','err']
abundance_data = [-99,0.8]
add_abundance_data(abundance_paper,abundance_norm,abundance_label,abundance_data)

found corresponding normalization data.
add paper related to abundance to table refs.
galid  1
create table  abu_table_2


In [43]:
#add abundance data
abundance_paper=['BeersFun',2008,'http://adsabs.harvard.edu/abs/2012ApJ...751..102V','SMC']
abundance_norm=['Anders','1989']
abundance_label=['[Fe/H]','err']
abundance_data = [-13,0.33]
add_abundance_data(abundance_paper,abundance_norm,abundance_label,abundance_data)

found corresponding normalization data.
abu data paper exists already. do nothing
galaxy name  SMC
galid  2
create table  abu_table_3


In [44]:
#as expected, the new table exists
abundance_paper=['BeersFun',2008,'Milky Way']
get_abundance_data(abu_paper=abundance_paper)

[(1, -99.0, 0.8)]


,abuid,Fe/H,err
0,1,-99.0,0.8


In [45]:
#check if table exists in abu_table_reg, the registration table. Yes, it exists as the second entry.
get_overview_abundance_tables()

In [46]:
get_galaxies()

,Galaxies
0,Milky Way
1,SMC
